In [1]:
%load_ext cython

In [2]:
import os
os.chdir("C:\\Users\\chris\\PycharmProjects\\hmm-master")
os.getcwd()

'C:\\Users\\chris\\PycharmProjects\\hmm-master'

In [3]:
import numpy as np
from utils.simulate_returns import simulate_2state_gaussian
from models.hmm_jump import JumpHMM

In [4]:
model = JumpHMM(n_states=2, random_state=42)
returns, true_regimes = simulate_2state_gaussian(plotting=False)  # Simulate some data from two normal distributions

Z = model.construct_features(returns, window_len=6)
model._init_params(Z)

l2_norms = model._l2_norm_squared(Z, model.theta)

In [5]:
%%timeit -r 10 -n 100
model._fit_state_seq(Z)

125 µs ± 15.4 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [6]:
%%timeit -r 10 -n 100
model._fit_state_seq1(Z)

21 ms ± 132 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [4]:
model = JumpHMM(n_states=2, random_state=42)
returns, true_regimes = simulate_2state_gaussian(plotting=False)  # Simulate some data from two normal distributions

Z = model.construct_features(returns, window_len=6)

In [5]:
%%timeit
model.fit(Z)

C:\Users\chris\PycharmProjects\hmm-master\models\hmm_jump.py:121: RuntimeWarning: divide by zero encountered in long_scalars
  self.theta[:, j] = 1 / N_j * z
C:\Users\chris\PycharmProjects\hmm-master\models\hmm_jump.py:121: RuntimeWarning: invalid value encountered in multiply
  self.theta[:, j] = 1 / N_j * z
C:\Users\chris\PycharmProjects\hmm-master\models\hmm_jump.py:121: RuntimeWarning: divide by zero encountered in long_scalars
  self.theta[:, j] = 1 / N_j * z
C:\Users\chris\PycharmProjects\hmm-master\models\hmm_jump.py:121: RuntimeWarning: invalid value encountered in multiply
  self.theta[:, j] = 1 / N_j * z
C:\Users\chris\PycharmProjects\hmm-master\models\hmm_jump.py:121: RuntimeWarning: divide by zero encountered in long_scalars
  self.theta[:, j] = 1 / N_j * z
C:\Users\chris\PycharmProjects\hmm-master\models\hmm_jump.py:121: RuntimeWarning: invalid value encountered in multiply
  self.theta[:, j] = 1 / N_j * z
C:\Users\chris\PycharmProjects\hmm-master\models\hmm_jump.py:121: R

125 ms ± 5.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
model.fit(Z)

In [6]:
%%timeit
model.fit1(Z)

859 ms ± 79.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## cython implementation

In [43]:
%%cython -3

cimport cython
from cython cimport view
from libc.math cimport exp, log, INFINITY, isinf
#from numpy.math cimport expl, logl, isinf, INFINITY

import numpy as np

ctypedef double dtype_t

@cython.boundscheck(False)
cdef inline int _argmin(dtype_t[:] X) nogil:
    cdef dtype_t X_min = INFINITY
    cdef int pos = 0
    cdef int i
    for i in range(X.shape[0]):
        if X[i] < X_min:
            X_min = X[i]
            pos = i
    return pos

@cython.boundscheck(False)
cdef inline dtype_t _min(dtype_t[:] X) nogil:
    return X[_argmin(X)]

@cython.boundscheck(False)
@cython.wraparound(False)
def jump_state_seq(int n_samples, int n_states, int n_features,
                   dtype_t jump_penalty,
                   dtype_t[:, :] l2_norms):

    cdef int i, j ,t, where_from
    cdef int[::view.contiguous] state_sequence = np.empty(n_samples, dtype=np.int32)
    cdef dtype_t[:, ::view.contiguous] losses = np.zeros(shape=(n_samples, n_states))
    
    # Temporary variables
    cdef dtype_t[::view.contiguous] state_change_penalty = np.empty(n_states)

    with nogil:
        losses[n_samples - 1] = l2_norms[n_samples - 1]
        
        # Backward recursion to compute losses
        for t in range(n_samples - 2, -1, -1):
            for i in range(n_states):            
                
                for j in range(n_states):
                    # If j==1, then no state change occurred and no jump penalty applies
                    if j == i:
                        state_change_penalty[j] = l2_norms[t+1, j]
                    else:
                        state_change_penalty[j] = l2_norms[t+1, j] + jump_penalty
                
                
                losses[t, i] = l2_norms[t, i] + _min(state_change_penalty)
                
        # Use losses in forward recursion to compute most likeley state sequence
        state_sequence[0] = _argmin(losses[0])
        
        for t in range(1, n_samples):
            for i in range(n_states):
                
                if i == state_sequence[t-1]:
                    state_change_penalty[i] = losses[t, i]
                else:
                    state_change_penalty[i] = losses[t, i] + jump_penalty
                
            state_sequence[t] = _argmin(state_change_penalty)
        
    return np.asarray(losses), np.asarray(state_sequence)

In [44]:
jump_state_seq(len(Z), 2, Z.shape[1],
               0.2,
               l2_norms)

(array([[0.02821312, 0.50028372],
        [0.02900698, 0.65667288],
        [0.04496802, 0.5991863 ],
        ...,
        [0.1234966 , 0.35511593],
        [0.06949626, 0.40281096],
        [0.04139661, 0.18216915]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [45]:
%%timeit
jump_state_seq(len(Z), 2, Z.shape[1],
               0.2,
               l2_norms)

32.4 µs ± 407 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
